In [1]:
import pandas as pd
import datetime as dt
from datetime import date,time,datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import pickle as pk
import os
import re
import nltk
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string

!pwd

data_dir = "./mimic-iii"
files_dir = "../dump_files"

pheno = pd.read_csv(os.path.join(files_dir, 'PHENOTYPE.csv'))

pheno

/home/akshara/coding/Phenotyping/preprocess


,HADM_ID,SUBJECT_ID,ROW_ID,ADVANCED.CANCER,ADVANCED.HEART.DISEASE,ADVANCED.LUNG.DISEASE,ALCOHOL.ABUSE,BATCH.ID,CHRONIC.NEUROLOGICAL.DYSTROPHIES,CHRONIC.PAIN.FIBROMYALGIA,DEMENTIA,DEPRESSION,DEVELOPMENTAL.DELAY.RETARDATION,NON.ADHERENCE,NONE,OBESITY,OPERATOR,OTHER.SUBSTANCE.ABUSE,SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS,UNSURE
0,100103,3365,17170,0,0,0,0,08Apr16,0,0,0,0,0,0,1,0,JTW,0,0,0
1,100103,3365,17170,0,0,0,0,08Apr16,0,0,0,0,0,0,1,0,ETM,0,0,0
2,100137,27290,50828,0,0,0,0,08Apr16,0,0,0,0,0,0,1,0,JF,0,0,0
3,100137,27290,50828,0,0,0,0,08Apr16,0,0,0,0,0,0,1,0,JW,0,0,0
4,100473,5525,52670,0,0,0,0,18Feb16,0,0,0,0,0,0,1,0,JW,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271,199807,23549,2787,0,1,1,0,07Dec15,0,0,0,0,0,0,0,0,ETM,0,0,0
2272,199883,7029,18731,0,0,0,1,18Feb16,0,0,0,0,0,1,0,0,JW,0,0,0
2273,199883,7029,18731,0,0,0,1,18Feb16,0,0,0,0,0,1,0,0,JF,0,0,0
2274,199884,1931,8180,0,1,0,0,16Mar16,0,0,0,1,0,0,0,0,DAG,0,0,0


In [2]:
pheno.columns

Index(['HADM_ID', 'SUBJECT_ID', 'ROW_ID', 'ADVANCED.CANCER',
       'ADVANCED.HEART.DISEASE', 'ADVANCED.LUNG.DISEASE', 'ALCOHOL.ABUSE',
       'BATCH.ID', 'CHRONIC.NEUROLOGICAL.DYSTROPHIES',
       'CHRONIC.PAIN.FIBROMYALGIA', 'DEMENTIA', 'DEPRESSION',
       'DEVELOPMENTAL.DELAY.RETARDATION', 'NON.ADHERENCE', 'NONE', 'OBESITY',
       'OPERATOR', 'OTHER.SUBSTANCE.ABUSE',
       'SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS', 'UNSURE'],
      dtype='object')

In [24]:
notes1 = pd.read_csv(os.path.join(files_dir, 'notes_disch.csv'))

df = pheno.merge(notes1, on='HADM_ID')

# pheno.rename(columns={'Hospital.Admission.ID':'HADM_ID'}, inplace=True)
df = notes1.merge(pheno, on='HADM_ID')

df

columns_titles = ['HADM_ID', 'TEXT', 'OBESITY', 'ADVANCED.HEART.DISEASE',
       'ADVANCED.LUNG.DISEASE',
       'SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS', 'ALCOHOL.ABUSE',
       'OTHER.SUBSTANCE.ABUSE', 'CHRONIC.PAIN.FIBROMYALGIA',
       'CHRONIC.NEUROLOGICAL.DYSTROPHIES', 'ADVANCED.CANCER', 'DEPRESSION'
       ]
df=df.reindex(columns=columns_titles)

df.columns

notes2 = pd.read_csv(os.path.join(files_dir, 'notes_phys.csv'))

df = df.merge(notes2, on='HADM_ID')

df.shape

df.columns

df = df[['HADM_ID', 'TEXT_x', 'TEXT_y','OBESITY', 'ADVANCED.HEART.DISEASE',
       'ADVANCED.LUNG.DISEASE',
       'SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS', 'ALCOHOL.ABUSE',
       'OTHER.SUBSTANCE.ABUSE', 'CHRONIC.PAIN.FIBROMYALGIA',
       'CHRONIC.NEUROLOGICAL.DYSTROPHIES', 'ADVANCED.CANCER', 'DEPRESSION'
       ]]

df.shape

df

df.to_csv(os.path.join(files_dir, 'pheno-disch-phys.csv'),index=False)

In [25]:
df.shape

(218, 13)

In [26]:
df

,HADM_ID,TEXT_x,TEXT_y,OBESITY,ADVANCED.HEART.DISEASE,ADVANCED.LUNG.DISEASE,SCHIZOPHRENIA.AND.OTHER.PSYCHIATRIC.DISORDERS,ALCOHOL.ABUSE,OTHER.SUBSTANCE.ABUSE,CHRONIC.PAIN.FIBROMYALGIA,CHRONIC.NEUROLOGICAL.DYSTROPHIES,ADVANCED.CANCER,DEPRESSION
0,100485.0,admission date discharge date date birth sex f...,cvicu hpi hd2 pod p cabg x3 lima lad svg om1 s...,0,0,0,1,0,0,0,0,0,1
1,100485.0,admission date discharge date date birth sex f...,cvicu hpi hd2 pod p cabg x3 lima lad svg om1 s...,0,0,0,1,0,0,0,0,0,1
2,102607.0,admission date discharge date date birth sex f...,chief complaint hd line infection saw examined...,0,0,0,0,0,0,1,1,0,0
3,102607.0,admission date discharge date date birth sex f...,chief complaint hd line infection saw examined...,0,0,0,0,0,0,0,0,0,0
4,103002.0,admission date discharge date date birth sex f...,chief complaint shortness breath hpi ms year o...,0,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,198393.0,admission date discharge date service medicine...,title chief complaint hour events cards follow...,0,0,0,0,0,0,0,0,0,0
214,198393.0,admission date discharge date service medicine...,title chief complaint hour events cards follow...,0,0,1,0,0,0,0,0,0,0
215,198393.0,admission date discharge date service medicine...,title chief complaint hour events cards follow...,0,0,0,0,1,0,0,0,0,0
216,198781.0,admission date discharge date date birth sex s...,title chief complaint hour events blood cultur...,0,0,0,0,0,0,0,0,0,0


In [16]:
notes2 = pd.read_csv(os.path.join(files_dir, 'pheno-disch-phys.csv'))

In [17]:
nurse = notes2.TEXT_x.tolist()
disch = notes2.TEXT_y.tolist()

In [18]:
nurse[0]

'admission date discharge date date birth sex f service cardiothoracic allergies penicillins percocet tetanus latex attending chief complaint chest tightness major surgical invasive procedure p coronary artery bypass graft surgery left internal mammary artery left anterior descending saphenous vein graft obtuse marginal saphenous vein graft obtuse marginal history present illness year old female scheduled distal sfa knee popliteal artery bypass relieve symptoms dr developed episode chest heaviness approximately weeks ago occurred sleeping lasted couple days resolved spontaneously past medical history diabetes mellitus type hypertension hyperlipidemia hypothyroidism depression osteopenia squampous cell cancer p excision renal tumor renal calculi bronchitis anxiety p cholecystectomy p appendectomy p polypectomy social history occupation retired hairstylist lives husband daughter grandson tobacco pack per day etoh rare alcohol use recreational drug use family history noncontributory physi

In [19]:
disch[0]

'cvicu hpi hd2 pod p cabg x3 lima lad svg om1 svg om2 ejection fraction hempglobin a1c pre op weight lbs kgs baseline creatinine foley day2 pmh dm type htn hyperlipidemia hypothyroidism depression osteopenia cholecystectomy appendectomy squamous cell cancer skin p excision renal tumor renal calculi two c sections bronchitis anxiety history polypectomy metformin glipizide simvastatin synthroid ativan prn bupropion clotrimazole foot asa current medications acetaminophen albuterol neb soln aspirin ec bupropion clotrimazole cream docusate sodium furosemide hydromorphone insulin levothyroxine sodium metoprolol tartrate morphine sulfate ranitidine simvastatin tramadol vancomycin hour events received line changed overwire access weaned sedation extubated without complications invasive ventilation start pm ekg pm nasal swab pm cordis introducer start pm arterial line start pm cordis introducer stop pm multi lumen start pm extubation pm complications invasive ventilation stop pm allergies penic

In [7]:
notes1 = pd.read_csv(os.path.join(files_dir, 'notes_nurse.csv'))

In [9]:
text = notes1.TEXT.tolist()

In [10]:
text[0]

'59 year old male hcv related cirrhosis grade ii esophageal varices portal gastropathy presenting two days coffee ground emesis streaks bright red well dark stools also endorses feeling lightheaded arrival ed bp hr rr ra given l ns iv protonix mg iv started octreotide drip guaiac positive brown stool rectal exam seen liver fellow ed felt unlikely variceal bleed recommended work infection ng tube attempted however patient unable tolerate ed abdominal ultrasound done showed patent portal vein scant ascites enough tap prior transfer micu blood pressure dropped gastrointestinal bleed upper melena gi bleed gib assessment bleeding noted hct stable units prbc vs stable weaned levophed gtt action received units prbc attempted place ngt failed gagging continuously gi following feel though esophageal bleed likely scope monday started dopamine gtt bp labile dopamine rij placement dopamine turned started levophed total l ivf given iv protonix response hct stable bp normalized maintaining levophed 